In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 2
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

# Epochs

In [9]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [10]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [11]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [12]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [13]:
perceptual_ext = PerceptualLoss()

In [14]:
sobel_filter = Sobel().to(device)

In [15]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data
        
        B, T, C, H, W = x.size()
        x = x.to(device).view(-1, C, H, W)
        y = y.to(device).view(-1, 1, H, W)
        air_x = air_x.to(device).view(-1, 1, H, W)
        bone_x = bone_x.to(device).view(-1, 1, H, W)

        y_pr = generator(x.float())
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
    
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y.float())
        for _y_cls in y_cls:
            _y_cls = _y_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _y_cls.size()
            total = _B
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(_B, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = _y_pr_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _y_pr_cls.size()
            total = _B
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(_B, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]

 
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  

        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        edge_y_cls = edge_discriminator(edge_y.float())
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = _edge_y_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _edge_y_cls.size()
            total = _B
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros((_B), dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = _edge_pr_cls.view(B, T, 2)[:, 1, :]
            _B, _C = _edge_pr_cls.size()
            total = _B
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones((_B), dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        

        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        x = x.view(B, T, C, H, W)[:, 1, 0].unsqueeze(1)
        y = y.view(B, T, 1, H, W)[:, 1, :]
        y_pr = y_pr.view(B, T, 1, H, W)[:, 1, :]
        air_x = air_x.view(B, T, 1, H, W)[:, 1, :]
        bone_x = bone_x.view(B, T, 1, H, W)[:, 1, :]
        
        # main
        _ssim = ssim(y.float(), y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y.float(), 1.0)
        _snr = SNR()(y_pr, y.float())
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window = AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
        
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]
        
        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [16]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    iid = iid
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data
        
        B, T, C, H, W = x.size()
        x = x.to(device).view(-1, C, H, W)
        y = y.to(device).view(-1, 1, H, W)
        air_x = air_x.to(device).view(-1, 1, H, W)
        bone_x = bone_x.to(device).view(-1, 1, H, W)

        y_pr = model(x.float())
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
 
        x = x.view(B, T, C, H, W)[:, 1, 0].unsqueeze(1)
        y = y.view(B, T, 1, H, W)[:, 1, :]
        air_x = air_x.view(B, T, 1, H, W)[:, 1, :]
        bone_x = bone_x.view(B, T, 1, H, W)[:, 1, :]
        y_pr = y_pr.view(B, T, 1, H, W)[:, 1, :]

        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)

        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window = AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0).float()
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)

            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                
            iid += 1

# Discriminator

In [17]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [18]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = smp.Unet(encoder_name="resnet18", in_channels=in_channel, classes=n_classes).encoder
        self.encoder = encoder
        self.fc = MultiScaleHeads(n_classes=n_classes, channels=(512, ), activation=None)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc([x[-1]])
        return x

# Generator

In [19]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [20]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [21]:
import glob
from codes.dataset import DicomSegmentDataset, DicomsSegmentDataset
import codes.augmentation as aug
from codes.temporal_shift import make_temporal_shift

In [22]:
# run_name = wandb.run.name
run_name = "ancient-glitter-294"
ELECTRON = False
G_COORD = True
L_COORD= False

N_SEGMENT = 3

In [23]:
VIEW_BOUND = (-500, 500)
AIR_BOUND = (-500, -499)
BONE_BOUND = (255, 256)
if ELECTRON:
    VIEW_BOUND = (0.5, 1.5)
    AIR_BOUND = (0.5, 0.5009)
    BONE_BOUND = (1.2, 1.2009)  

In [24]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [25]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
make_temporal_shift(model, N_SEGMENT, n_div=4, place="blockres")

model.load_state_dict(checkpoint["model"])

=> n_segment per stage: [3, 3, 3, 3]
=> Using n_round 2 to insert temporal shift
=> Processing stage with 3 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 8 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 36 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 3 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

In [26]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
make_temporal_shift(tf_discriminator, N_SEGMENT, n_div=4, place="blockres")

tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

=> n_segment per stage: [3, 3, 3, 3]
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

In [27]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
make_temporal_shift(edge_discriminator, N_SEGMENT, n_div=4, place="blockres")

edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

=> n_segment per stage: [3, 3, 3, 3]
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4
=> Processing stage with 2 blocks residual
=> Using fold div: 4
=> Using fold div: 4


<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [28]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [29]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [30]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [31]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:13,  2.05it/s]
27it [00:11,  2.45it/s]
26it [00:10,  2.43it/s]
28it [00:11,  2.49it/s]
27it [00:11,  2.39it/s]
28it [00:11,  2.51it/s]
27it [00:10,  2.47it/s]


In [32]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:45,  4.20it/s]


(0.8309638993278223,
 23.251653890959254,
 14.255925842604713,
 0.027967296092141984,
 0.9964873279352924,
 0.8959748074097442,
 0.3639968740534408,
 0.5052356020942408,
 0.5026178010471204)

## Pelvic Test CT

In [33]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [34]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [35]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [36]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:10,  2.71it/s]
27it [00:10,  2.60it/s]
26it [00:10,  2.57it/s]
28it [00:11,  2.51it/s]
27it [00:11,  2.32it/s]
28it [00:11,  2.36it/s]
27it [00:11,  2.34it/s]


In [37]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:46,  4.11it/s]


(0.9221871801071766,
 27.950586209122424,
 18.954858220684592,
 0.016201834998907844,
 0.9965354822069851,
 0.8860778349164444,
 0.3771585191731678,
 0.5,
 0.5052356020942408)

## Pelvic L1 CBCT

In [38]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [39]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [40]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [41]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

29it [00:12,  2.36it/s]
26it [00:11,  2.34it/s]
28it [00:12,  2.33it/s]
28it [00:12,  2.27it/s]
27it [00:11,  2.30it/s]
26it [00:11,  2.27it/s]
27it [00:11,  2.28it/s]
30it [00:13,  2.29it/s]
28it [00:12,  2.27it/s]
29it [00:12,  2.35it/s]
29it [00:12,  2.35it/s]
27it [00:11,  2.30it/s]
28it [00:12,  2.30it/s]
26it [00:11,  2.29it/s]
28it [00:12,  2.31it/s]
28it [00:11,  2.33it/s]
27it [00:11,  2.32it/s]
26it [00:10,  2.38it/s]
27it [00:11,  2.37it/s]
29it [00:12,  2.34it/s]
30it [00:12,  2.34it/s]
32it [00:13,  2.38it/s]
28it [00:11,  2.34it/s]
28it [00:11,  2.35it/s]
15it [00:06,  2.26it/s]
28it [00:11,  2.36it/s]
26it [00:11,  2.28it/s]
28it [00:12,  2.31it/s]
26it [00:11,  2.34it/s]
27it [00:11,  2.33it/s]
36it [00:15,  2.33it/s]
28it [00:12,  2.32it/s]
27it [00:11,  2.30it/s]
28it [00:11,  2.37it/s]
30it [00:12,  2.34it/s]
27it [00:11,  2.43it/s]
26it [00:11,  2.32it/s]
29it [00:12,  2.38it/s]
27it [00:10,  2.57it/s]
27it [00:11,  2.30it/s]
28it [00:12,  2.29it/s]
26it [00:10,  2.

In [42]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [05:48,  4.19it/s]


(0.8165491476972052,
 22.62021813690458,
 13.419200422343279,
 0.030679134395057893,
 0.9960477455854673,
 0.8631159825582547,
 0.39131508309358326,
 0.5048043925875085,
 0.5054907343857241)

## Pelvic L1 CT

In [43]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [44]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [45]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [46]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

29it [00:11,  2.47it/s]
26it [00:10,  2.37it/s]
28it [00:11,  2.41it/s]
28it [00:11,  2.44it/s]
27it [00:12,  2.25it/s]
26it [00:11,  2.23it/s]
27it [00:12,  2.22it/s]
30it [00:13,  2.24it/s]
28it [00:12,  2.23it/s]
29it [00:12,  2.23it/s]
29it [00:11,  2.50it/s]
27it [00:12,  2.22it/s]
28it [00:12,  2.25it/s]
26it [00:11,  2.25it/s]
28it [00:12,  2.23it/s]
28it [00:12,  2.29it/s]
27it [00:12,  2.23it/s]
26it [00:11,  2.21it/s]
27it [00:12,  2.21it/s]
29it [00:12,  2.26it/s]
30it [00:13,  2.27it/s]
32it [00:14,  2.24it/s]
28it [00:12,  2.32it/s]
28it [00:12,  2.32it/s]
15it [00:06,  2.27it/s]
28it [00:11,  2.34it/s]
26it [00:11,  2.28it/s]
28it [00:12,  2.32it/s]
26it [00:11,  2.30it/s]
27it [00:11,  2.30it/s]
36it [00:15,  2.37it/s]
28it [00:12,  2.30it/s]
27it [00:11,  2.32it/s]
28it [00:12,  2.33it/s]
30it [00:12,  2.35it/s]
27it [00:11,  2.31it/s]
26it [00:11,  2.29it/s]
29it [00:12,  2.36it/s]
27it [00:11,  2.38it/s]
27it [00:11,  2.35it/s]
28it [00:11,  2.36it/s]
26it [00:11,  2.

In [47]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [05:42,  4.26it/s]


(0.9221783503736628,
 27.917769538758957,
 18.716751873615923,
 0.01699857852008135,
 0.9964787485975161,
 0.8676638664713234,
 0.41221397391065967,
 0.5,
 0.5037748798901853)

## Pelvic L2 CBCT

In [48]:
test_case_path = 'L2_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [49]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [50]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_test"))
except FileExistsError:
    pass

In [51]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_test", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_test", patient_id)) 
    iid += len(scans)

31it [00:12,  2.40it/s]
27it [00:11,  2.44it/s]
28it [00:11,  2.42it/s]
27it [00:11,  2.38it/s]
27it [00:11,  2.39it/s]
26it [00:10,  2.38it/s]
30it [00:12,  2.39it/s]
27it [00:11,  2.34it/s]
29it [00:12,  2.39it/s]
26it [00:11,  2.34it/s]
26it [00:10,  2.58it/s]
27it [00:10,  2.57it/s]
26it [00:10,  2.47it/s]
27it [00:10,  2.54it/s]
26it [00:10,  2.45it/s]
30it [00:12,  2.33it/s]
26it [00:11,  2.29it/s]
26it [00:11,  2.34it/s]
26it [00:10,  2.43it/s]
27it [00:11,  2.34it/s]
27it [00:11,  2.34it/s]
26it [00:11,  2.26it/s]
29it [00:12,  2.35it/s]
29it [00:12,  2.33it/s]


In [52]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

656it [02:39,  4.11it/s]


(0.798168907078301,
 21.68183240367145,
 12.136475219868306,
 0.034471193904897605,
 0.996084503722364,
 0.8621848350311677,
 0.43770451978893904,
 0.5015243902439024,
 0.510670731707317)

## Pelvic L2 CT

In [53]:
test_case_path = 'L2_pelvic_processed/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [54]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [55]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [56]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

31it [00:13,  2.31it/s]
27it [00:11,  2.30it/s]
28it [00:11,  2.37it/s]
27it [00:11,  2.31it/s]
27it [00:11,  2.33it/s]
26it [00:11,  2.32it/s]
30it [00:12,  2.31it/s]
27it [00:11,  2.30it/s]
29it [00:12,  2.30it/s]
26it [00:11,  2.33it/s]
26it [00:11,  2.33it/s]
27it [00:11,  2.30it/s]
26it [00:11,  2.32it/s]
27it [00:11,  2.31it/s]
26it [00:11,  2.32it/s]
30it [00:12,  2.32it/s]
26it [00:11,  2.32it/s]
26it [00:11,  2.30it/s]
26it [00:11,  2.30it/s]
27it [00:11,  2.35it/s]
27it [00:11,  2.35it/s]
26it [00:11,  2.28it/s]
29it [00:12,  2.32it/s]
29it [00:12,  2.34it/s]


In [57]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

656it [02:35,  4.22it/s]


(0.9271652929303122,
 28.75937658693732,
 19.214019346527937,
 0.014122129449189254,
 0.995712746017202,
 0.8881252846922221,
 0.3895019536906081,
 0.49771341463414637,
 0.506859756097561)

# Abdomen

In [58]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [59]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [60]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [61]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

28it [00:12,  2.33it/s]
78it [00:32,  2.38it/s]
50it [00:21,  2.36it/s]
29it [00:12,  2.33it/s]
50it [00:21,  2.32it/s]
72it [00:30,  2.37it/s]
39it [00:16,  2.35it/s]


In [62]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

346it [01:23,  4.13it/s]


(0.7237869368467716,
 20.16665609447942,
 10.699903218732404,
 0.04567684529273841,
 0.9895321600815633,
 0.7719602876959686,
 0.3531258051429009,
 0.43352601156069365,
 0.5794797687861272)

## Abdomen on CT

In [63]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [64]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [65]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

28it [00:11,  2.36it/s]
78it [00:32,  2.38it/s]
50it [00:21,  2.36it/s]
29it [00:12,  2.36it/s]
50it [00:21,  2.37it/s]
72it [00:29,  2.44it/s]
39it [00:16,  2.39it/s]


In [66]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="abdomen", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

346it [01:27,  3.94it/s]


(0.8934159807731651,
 26.694622872192735,
 17.227870034344622,
 0.020012063367259986,
 0.9892666536279353,
 0.7887182986607489,
 0.34500219880716304,
 0.3901734104046243,
 0.5664739884393064)

# Chest

In [67]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [68]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [69]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [70]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

49it [00:21,  2.23it/s]
39it [00:17,  2.22it/s]
40it [00:17,  2.23it/s]
29it [00:13,  2.14it/s]
35it [00:16,  2.18it/s]
37it [00:16,  2.23it/s]
34it [00:15,  2.20it/s]


In [71]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [01:03,  4.15it/s]


(0.7481916784786906,
 20.29864187929567,
 9.850771158820322,
 0.03987785061191017,
 0.9824015778274002,
 0.8134177358500576,
 0.472852885723114,
 0.48859315589353614,
 0.5057034220532319)

## Chest on CT

In [72]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [73]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [74]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="chest",segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

49it [00:21,  2.32it/s]
39it [00:16,  2.34it/s]
40it [00:17,  2.34it/s]
29it [00:12,  2.30it/s]
35it [00:15,  2.27it/s]
37it [00:15,  2.33it/s]
34it [00:15,  2.26it/s]


In [75]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="chest", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [01:02,  4.18it/s]


(0.9193297175853424,
 27.61429377653753,
 17.166423112267324,
 0.015510161060259143,
 0.9808539239033841,
 0.787555376193136,
 0.4231113088051629,
 0.36311787072243346,
 0.5038022813688213)

# Headneck

In [76]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [77]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [78]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [79]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

58it [00:24,  2.37it/s]
55it [00:23,  2.35it/s]
55it [00:23,  2.36it/s]
56it [00:22,  2.50it/s]
56it [00:22,  2.50it/s]
55it [00:23,  2.35it/s]
56it [00:23,  2.38it/s]


In [80]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [01:32,  4.22it/s]


(0.8644961258944344,
 21.170285120034766,
 7.217732420967668,
 0.027951280007619993,
 0.8204033668837382,
 0.8212641206078567,
 0.2288322842887143,
 0.5089514066496164,
 0.6777493606138107)

## Headneck on CT

In [81]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [82]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [83]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomSegmentDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

58it [00:24,  2.39it/s]
55it [00:23,  2.35it/s]
55it [00:23,  2.36it/s]
56it [00:23,  2.34it/s]
56it [00:23,  2.35it/s]
55it [00:23,  2.37it/s]
56it [00:23,  2.35it/s]


In [84]:
testset = DicomsSegmentDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="headneck", segment=N_SEGMENT, g_coord=G_COORD, l_coord=L_COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [01:33,  4.17it/s]


(0.9143856779083877,
 25.9915285500724,
 12.038975719295804,
 0.016055005940768266,
 0.8000978753554535,
 0.8077351491387996,
 0.2334904273776228,
 0.4820971867007673,
 0.6112531969309463)